In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
import statsmodels.api as sm
import warnings

warnings.filterwarnings('ignore')


In [4]:
# 1. Data Loading & Inspection
# ---------------------------

# Load CSV
df = pd.read_csv('CHNADIGARH DATA.csv')

In [5]:
# Inspect
print("\n--- Dataset Head ---")
print(df.head())

print("\n--- Dataset Info ---")
print(df.info())

print("\n--- Summary Statistics ---")
print(df.describe())

print("\n--- Missing Values Before Cleaning ---")
print(df.isnull().sum())


--- Dataset Head ---
            To Date   PM2.5    PM10     NO    NO2    NOx    NH3   SO2    CO  \
0  03-01-2024 00:00   95.77  142.61   3.25  41.37  23.67  22.18  8.27  1.18   
1  04-01-2024 00:00  123.76  189.76  20.27  34.17  31.85  22.90  2.84  1.44   
2  05-01-2024 00:00   77.04  115.82  13.17  28.34  25.14  26.32  0.92  1.20   
3  06-01-2024 00:00  115.04  167.34  10.34  31.36  24.80  34.99  3.16  1.36   
4  07-01-2024 00:00  133.54  183.38  13.61  31.42  27.28  35.38  1.16  1.42   

   Ozone  ...  Toluene  Eth-Benzene  MP-Xylene  O-Xylene     RH    WS      WD  \
0  22.59  ...    12.35         3.09       2.55       NaN  87.25  0.51  226.29   
1  32.64  ...    20.69         5.88       5.43       NaN  86.27  0.37  205.25   
2  17.19  ...    16.58         5.28       4.37       NaN  78.58  0.48  288.74   
3  15.98  ...    13.41         4.32       3.78       NaN  70.66  0.40  256.57   
4  18.34  ...    16.28         5.84       4.67       NaN  74.05  0.44  257.42   

      SR  Xylene

In [21]:

# 2. Data Cleaning
# ---------------------------

# Drop rows where PM2.5 (target) is missing
df = df.dropna(subset=['PM2.5'])
# Fill any remaining missing feature values with median (more robust)
features = features.fillna(features.median())


# Fill remaining numeric missing values with median
df.fillna(df.median(numeric_only=True), inplace=True)

# Features for models (exclude targets)
features = df.select_dtypes(include=[np.number]).drop(
    ['PM2.5', 'PM2.5_Category'], axis=1, errors='ignore'
)

# Fill missing feature values with median (instead of dropping)
features = features.fillna(features.median())

# Align targets with cleaned features
X_reg = features
y_reg = df.loc[features.index, 'PM2.5']
y_clf = df.loc[features.index, 'PM2.5_Category']


In [22]:
# 3. Setup Columns (Change as needed)
# ---------------------------

regression_target = 'PM2.5'                  # Continuous target for regression
classification_target = 'PM2.5_Category'     # Derived category from PM2.5 levels
time_series_variable = 'PM2.5'               # Forecast PM2.5 over time

# Features for ML models
features = df.select_dtypes(include=[np.number]).drop(
    [regression_target, classification_target], axis=1, errors='ignore'
)

# ---------------------------
# 3. Feature Engineering
# ---------------------------

# Classification Target: PM2.5 Category
def classify_pm25(value):
    if value <= 60:
        return 0  # Good
    elif value <= 120:
        return 1  # Moderate
    else:
        return 2  # Poor

df['PM2.5_Category'] = df['PM2.5'].apply(classify_pm25)

# Features for models (exclude targets)
features = df.select_dtypes(include=[np.number]).drop(
    ['PM2.5', 'PM2.5_Category'], axis=1, errors='ignore'
)

# Drop any rows with NaNs in features (final cleanup)
features = features.dropna()

# Align targets with cleaned features
X_reg = features
y_reg = df.loc[features.index, 'PM2.5']
y_clf = df.loc[features.index, 'PM2.5_Category']

# ---------------------------

In [23]:
# 4. Regression Models
# ---------------------------

print("\n--- Running Regression Models ---")

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
y_pred_lr = lr_model.predict(X_test_scaled)
mse_lr = mean_squared_error(y_test, y_pred_lr)

# Polynomial Regression (degree=2)
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)
y_pred_poly = poly_model.predict(X_test_poly)
mse_poly = mean_squared_error(y_test, y_pred_poly)

print(f"Linear Regression MSE: {mse_lr:.4f}")
print(f"Polynomial Regression (degree 2) MSE: {mse_poly:.4f}")



--- Running Regression Models ---


<class 'ValueError'>: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.